In [ ]:
import gym
from gym import wrappers
import matplotlib.pyplot as plt
from IPython import display
import random
import time
st = time.time()
import cv2
import pyNN.spiNNaker as p
p.setup(timestep=20)
import math

In [ ]:
TRAIN = True

In [ ]:
def inputFrameProcessor(frame):
    return cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
def inputToSpikeRateArray(frame):
    grayframe = inputFrameProcessor(frame) 
    grayframe= cv2.resize(grayframe,[84,84])
    return [(m+1) for line in grayframe for m in line]
def selectOutput(records,previous_spikes):
    choice = 0
    best = 0
    j=-1
    new_previous_spikes=[]
    if previous_spikes==[]:
        previous_spikes=[0]*len(records)
    for rec in records:
        j+=1
        #print("action",j,"spikes",len(rec)-previous_spikes[j])
        new_previous_spikes.append(len(rec))
    for i in range(len(records)):
        if (len(records[i])-previous_spikes[i])>best:
            best = len(records[i])-previous_spikes[i]
            choice = i
    return choice,new_previous_spikes

In [ ]:
env = gym.make("Pong-v0")
print("The environment has the following {} actions: {}".format(env.action_space.n, 
                                                                env.unwrapped.get_action_meanings()))
env = wrappers.Monitor(env, "/tmp/Pong-v0", force=True)
print(len(env.unwrapped.get_action_meanings()))

In [ ]:
##### INPUT LAYER #####
inputLayer = p.Population(len(inputToSpikeRateArray(env.render(mode='rgb_array'))),p.SpikeSourcePoisson(rate=inputToSpikeRateArray(env.render(mode='rgb_array'))))
inputLayer.record(["spikes"])
##### Training INPUT ######
if TRAIN:
    rewardLayer = [p.Population(1,p.SpikeSourcePoisson(rate=1)) for i in range(env.action_space.n)]
    punishmentLayer = [p.Population(1,p.SpikeSourcePoisson(rate=1)) for i in range(env.action_space.n)]
    
#####   STDP    #####
timing_rule = p.SpikePairRule(tau_plus=0.1, tau_minus=0.1, A_plus=0.1, A_minus=0.1)
weight_rule = p.AdditiveWeightDependence(w_max=10.0, w_min=0.01)
stdp_model_excitatory = p.STDPMechanism(timing_dependence=timing_rule, weight_dependence=weight_rule, weight=5)
#### SECOND LAYER ####
pop=[p.Population(int(100),p.IF_curr_exp()) for action in range(env.action_space.n)]
s = [m.record(["spikes"]) for m in pop]
#### Projections ####
projections01= [p.Projection(inputLayer,m,p.AllToAllConnector(), synapse_type=stdp_model_excitatory) for m in pop]
rewardproj=[]
punishmentproj = []
for i in range(env.action_space.n):
    rewardproj.append(p.Projection(rewardLayer[i],pop[i],p.AllToAllConnector(),synapse_type=p.StaticSynapse(weight=0.11)))
    punishmentproj.append(p.Projection(punishmentLayer[i],pop[i],p.AllToAllConnector(), synapse_type=p.StaticSynapse(weight=-50)))



In [ ]:
###RUN GAME TO CHECK IF ANYTHING IS ACHEIVED
rwrd_running_tot=[]
count=0
for i in range(100):
    step = 0
    total_reward = 0
    previous_spikes=[]
    action=0
    done =False
    env.reset()
    #for i in range(500):
    #    action = random.randint(0,5)
    #    env.step(action)
    reward=0
    #fig = plt.figure(figsize=(5, 5))
    #img = plt.imshow(env.render(mode='rgb_array'))
    while not done:
        count+=1
        print(count)
        #img.set_data(env.render(mode='rgb_array'))
        #display.display(plt.gcf())
        #display.clear_output(wait=True)
        #env.render()
        ri = total_reward
        
        for i in range(10):
            if not done:
                observation, reward, done, info = env.step(action)
        
        total_reward+=reward
        inputLayer.set(rate=inputToSpikeRateArray(observation))

        #p.run_until(10*i)
        #print("Action:############################################ ",action)
        if reward > 0:#REWARD SIGNAL CHANGES WHEN REWARD CHANGES
            for n in range(len(rewardLayer)):
                if n==action:
                    rewardLayer[n].set(rate = ((total_reward-ri)+1)*10)
                else:
                    rewardLayer[n].set(rate = 1)
        else:
            for n in range(len(punishmentLayer)):
                if n==action:
                    punishmentLayer[n].set(rate = (ri-total_reward)*10)
                else:
                    punishmentLayer[n].set(rate = 1)

        total_reward += reward
        action,previous_spikes = selectOutput([m.get_data("spikes").segments[0].spiketrains[0] for m in pop],previous_spikes)
        p.run_until(20*count)
        #action,previous_spikes = selectOutput([m.get_data("spikes").segments[0].spiketrains[0] for m in pop],previous_spikes)
    print("GAME REWARD",total_reward)
    display.clear_output(wait=True)
    #print("Total Reward",total_reward)
    rwrd_running_tot.append(total_reward)
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot([i for i in range(len(rwrd_running_tot))],rwrd_running_tot)
plt.savefig('plot.png', dpi=300, bbox_inches='tight')


In [ ]:
print(rwrd_running_tot)

In [ ]:
print(time.time()-st)